In [1]:
import re
import os
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import numpy as np
import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

# ini_path = 'E:/insper'
# files = 'datajud.csv'
# df = pd.read_csv(os.path.join(ini_path,files))
# interesse = set(list(df[df.ano_cnj>=2020].reset_index(drop = True).numero))

ini_path = 'D:/insper'
files = 'sentencas_todas_processadas.csv'
df = pd.read_csv(os.path.join(ini_path,files))
df['ano'] = df.numero_processo.str[11:15].astype(float)
df = df[df['Acórdão']=='Sim']
df.head()

C:\Users\henri\AppData\Local\Temp\ipykernel_8048\2464325058.py:22: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(ini_path,files))


,numero_processo,publicacao,numeros_paginas,nomes_pastas,estado,tipos_processuais,comarcas,representantes,assuntos,tribunal,...,mes,ano,nome_documento,data_decisao,orgao_julgador,tipo_publicacao,concedido,negado,Acórdão,decisao_clean
18,0644136-52.2017.8.04.0001,ADV: MARCELO AUGUSTO JANSEN SODRÉ (OAB 11282/ ...,314,02-08-2018,AM,ação civil de improbidade administrativa,Manaus,['11282/ AM'],dano ao erário,NaN,...,NaN,2017.0,NaN,NaN,NaN,NaN,RECURSO CONHECIDO E IMPRÓVIDO .,NaN,Sim,august jansen sodr improb administr dan erari ...
30,0215732-22.2018.8.04.0001,"ADV: SHELBY MOREIRA FINICELLI (OAB 5684/AM), A...",209,03-08-2018,AM,restituição de coisas apreendidas,Manaus,"['5684/AM', '8457/ AM']",coisas,NaN,...,NaN,2018.0,NaN,NaN,NaN,NaN,"Ante o exposto, em dissonância com o parecer M...",NaN,Sim,shelby mor finicell claud batist carvalh queir...
38,0630524-47.2017.8.04.0001,01 Processo: 0630524-47.2017.8.04.0001 - Apela...,18,04-04-2018,AM,apelação,Manaus,['8245/AM'],NaN,NaN,...,NaN,2017.0,NaN,NaN,NaN,NaN,NaN,"DECISÃO: ""Vistos, relatados e discutidos estes...",Sim,apelaca apel tailson albuquerqu hermerson rodr...
46,0623661-75.2017.8.04.0001,"ADV: ARNOLDO BENTES COIMBRA (OAB 345/AM), ADV:...",104,04-10-2018,AM,embargos de declaração,Manaus,"['345/AM', '6733/AM']",licitações,NaN,...,NaN,2017.0,NaN,NaN,NaN,NaN,NaN,"ADV: ARNOLDO BENTES COIMBRA (OAB 345/AM), ADV:...",Sim,arnold bent coimbr daniel andrad mand seguranc...
53,0035708-44.2010.8.04.0012,"ADV: EDNILSON PIMENTEL MATOS (OAB 1799/AM), RO...",109,05-03-2018,AM,ação civil de improbidade administrativa,NaN,"['1799/AM', '4314/AM', '3665/AM', '4714/AM']",improbidade administrativa,NaN,...,NaN,2010.0,NaN,NaN,NaN,NaN,(STJ - EDcl no REsp: 1635464 MS 2015/0277170-4...,NaN,Sim,ednilson pimentel mat roberv mend josaf mor ka...


In [2]:
tjs = ['TJBA', 'TJCE', 'TJMS', 'TJMT', 'TJRJ', 'TJRS', 'TJSC','TJSP']
df_final = df[df.tribunal.isin(tjs)]

# df['numero_limpo'] = df.numero_processo.str.replace('\.|\-', '')
# df_true = df[df.numero_limpo.isin(interesse)]
# df_true['result'] = 1
# df_false = df[~df.numero_limpo.isin(interesse)]
# df_false['result'] = 0

# df_final = pd.concat([df_true, df_false], ignore_index=True, sort=False)
# df_final.to_excel(os.path.join(ini_path,'todas_sentencas.xlsx'))
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_final['decisao_clean'], df_final['result'], random_state=42)

# for tribunal in tjs:
lista_palavras = list()
df_tribunal = df_final[df_final.tribunal == 'TJSP']
train_x = df_tribunal.decisao_clean
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=10000)
tfidf_vect_ngram.fit(df_final[df_final.tribunal != 'TJSP']['decisao_clean'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
df = pd.DataFrame(xtrain_tfidf_ngram.toarray(), columns=tfidf_vect_ngram.get_feature_names())
total_casos = list(range(0,len(df)))
for coluna in total_casos:
    lista_palavras.append(list(df.transpose().sort_values(by=coluna, ascending=False).head(30).index))
df_tribunal['termos'] = lista_palavras
df_tribunal.to_excel(os.path.join(ini_path,f'dados_TJSP_termos.xlsx'))

d:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\henri\AppData\Local\Temp\ipykernel_8048\3902040932.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tribunal['termos'] = lista_palavras


In [ ]:

train_x = df_final['decisao_clean']
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=10000)
tfidf_vect_ngram.fit(df_final.decisao_clean)
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
# xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [9]:
df = pd.DataFrame(xtrain_tfidf_ngram.toarray(), columns=tfidf_vect_ngram.get_feature_names())
len(df)

d:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


43631

In [10]:
total_casos = list(range(0,len(df)))
lista_palavras = list()
for coluna in total_casos:
    lista_palavras.append(list(df.transpose().sort_values(by=coluna, ascending=False).head(30).index))
lista_palavras

[['duraca',
  'razo',
  'razoa',
  'principi',
  'reclamaca',
  'contrarrazo',
  'muit',
  'constituc',
  'efetiv',
  'respeit',
  'complex',
  'grau',
  'jog',
  'norm',
  'apresent razo',
  'cenari',
  'tjb',
  'apresent contrarrazo',
  'cert',
  'nobr',
  'clausul',
  'celer',
  'paragraf',
  'dupl',
  'apresent',
  'reserv',
  'sobretud',
  'posicion',
  'apelaca',
  'principi constituc'],
 ['bah',
  'licitaca',
  'ministeri publ bah',
  'publ bah',
  'justin',
  'prelimin',
  'passiv',
  'improb',
  'contrat',
  'mat merit',
  'locaca',
  'prefeit',
  'administr',
  'prelimin argu',
  'improb administr',
  'indici sufici',
  'manifestaca',
  'petica',
  'merit',
  'indici',
  'argu',
  'publ',
  'litisconsorci passiv',
  'afinal',
  'receb',
  'municipi',
  'net',
  'litisconsorci',
  'prelimin ilegitim passiv',
  'acion'],
 ['hug',
  'salvadorb',
  'des baix',
  'castel',
  'formul absolv',
  'dour',
  'tell',
  'lim',
  'ministeri publ bah',
  'publ bah',
  'robert',
  'improced

In [16]:
len(lista_palavras)

26249

In [ ]:
df_final['termos'] = lista_palavras
df_final.to_excel(os.path.join(ini_path,'dados_tjs_termos.xlsx'))
